In [1]:
import torch
import os
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())
print(torch.cuda.current_device())
torch.cuda.set_device(0)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# os.environ["CUDA_VISIBLE_DEVICES"] = '0' #disable CUDA by setting CUDA_VISIBLE_DEVICES to an empty string 


True
1
NVIDIA GeForce GTX 1660 Ti
0


In [2]:
def run_exp():
    if args.is_training:
        for ii in range(args.itr):
            # setting record of experiments
            exp = Exp(args)  # set experiments
            setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
                args.task_name,
                args.model_id,
                args.model,
                args.data,
                args.features,
                args.seq_len,
                args.label_len,
                args.pred_len,
                args.d_model,
                args.n_heads,
                args.e_layers,
                args.d_layers,
                args.d_ff,
                args.factor,
                args.embed,
                args.distil,
                args.des, ii)
    
            print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
            exp.train(setting)
    
            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.test(setting)
            torch.cuda.empty_cache()

In [3]:
import argparse
import os
import torch
from exp.exp_classification import Exp_Classification
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

parser = argparse.ArgumentParser(description='TimesNet')

# basic config
parser.add_argument('--task_name', type=str, required=True, default='long_term_forecast',
                    help='task name, options:[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
parser.add_argument('--is_training', type=int, required=True, default=1, help='status')
parser.add_argument('--model_id', type=str, required=True, default='test', help='model id')
parser.add_argument('--model', type=str, required=True, default='Autoformer',
                    help='model name, options: [Autoformer, Transformer, TimesNet]')

# data loader
parser.add_argument('--data', type=str, required=True, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')
parser.add_argument('--seasonal_patterns', type=str, default='Monthly', help='subset for M4')
parser.add_argument('--inverse', action='store_true', help='inverse output data', default=False)

# inputation task
parser.add_argument('--mask_rate', type=float, default=0.25, help='mask ratio')

# anomaly detection task
parser.add_argument('--anomaly_ratio', type=float, default=0.25, help='prior anomaly ratio (%)')

# model define
parser.add_argument('--top_k', type=int, default=5, help='for TimesBlock')
parser.add_argument('--num_kernels', type=int, default=6, help='for Inception')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=512, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.1, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--channel_independence', type=int, default=0,
                    help='1: channel dependence 0: channel independence for FreTS model')
# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='MSE', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')

# de-stationary projector params
parser.add_argument('--p_hidden_dims', type=int, nargs='+', default=[128, 128],
                    help='hidden layer dimensions of projector (List)')
parser.add_argument('--p_hidden_layers', type=int, default=2, help='number of hidden layers in projector')

# args = parser.parse_args()
args = dotdict()

# basic config
args.task_name= 'classification'
args.is_training= 1
args.model_id= 'test'
args.model= 'TimesNet'

# data loader
args.data = 'UEA'
args.root_path = './dataset/commodity/coffee/'
args.data_path ='coffee' 
args.features = 'M'
args.target = 'OT'
args.freq = 'h'
args.checkpoints = './checkpoints/'

# forecasting task
args.seq_len= 96
args.label_len= 14
args.pred_len= 96
args.seasonal_patterns= 'Monthly'
args.inverse= False

# imputation task
args.mask_rate= 0.25

# anomaly detection task
args.anomaly_ratio= 0.25

# model define
args.top_k= 2
args.num_kernels= 4
args.enc_in= 7
args.dec_in= 7
args.c_out= 7
args.d_model= 512 # changed 512
args.n_heads= 8
args.e_layers= 2
args.d_layers= 32
args.d_ff= 16 # changed from 2048
args.moving_avg= 25
args.factor= 1 # changed from 1 
args.distil= True
args.dropout= 0.05
args.embed= 'timeF'
args.activation= 'gelu'
args.output_attention= False
args.channel_independence= 0

# optimization
args.num_workers= 10
args.itr= 1
args.train_epochs= 100
args.batch_size= 8
args.patience= 3
args.learning_rate= 0.0005
args.des= 'test'
args.loss= 'MSE'
args.lradj= 'type1'
args.use_amp= False

# GPU
args.use_gpu= True
args.gpu= 0
args.use_multi_gpu= True
args.devices= '0,1,2,3'

# de-stationary projector param
p_hidden_dims= [128, 128]
p_hidden_layers= 2


print('Args in experiment:')
print(args)

if args.task_name == 'long_term_forecast':
    Exp = Exp_Long_Term_Forecast
elif args.task_name == 'short_term_forecast':
    Exp = Exp_Short_Term_Forecast
elif args.task_name == 'imputation':
    Exp = Exp_Imputation
elif args.task_name == 'anomaly_detection':
    Exp = Exp_Anomaly_Detection
elif args.task_name == 'classification':
    Exp = Exp_Classification
else:
    Exp = Exp_Long_Term_Forecast


Args in experiment:
{'task_name': 'classification', 'is_training': 1, 'model_id': 'test', 'model': 'TimesNet', 'data': 'UEA', 'root_path': './dataset/commodity/coffee/', 'data_path': 'coffee', 'features': 'M', 'target': 'OT', 'freq': 'h', 'checkpoints': './checkpoints/', 'seq_len': 96, 'label_len': 14, 'pred_len': 96, 'seasonal_patterns': 'Monthly', 'inverse': False, 'mask_rate': 0.25, 'anomaly_ratio': 0.25, 'top_k': 2, 'num_kernels': 4, 'enc_in': 7, 'dec_in': 7, 'c_out': 7, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 32, 'd_ff': 16, 'moving_avg': 25, 'factor': 1, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'channel_independence': 0, 'num_workers': 10, 'itr': 1, 'train_epochs': 100, 'batch_size': 8, 'patience': 3, 'learning_rate': 0.0005, 'des': 'test', 'loss': 'MSE', 'lradj': 'type1', 'use_amp': False, 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': True, 'devices': '0,1,2,3'}


In [ ]:
run_exp()

Use GPU: cuda:0
TRAIN
2010
TEST
503
>>>>>>>start training : classification_test_TimesNet_UEA_ftM_sl4_ll14_pl0_dm512_nh8_el2_dl32_df16_fc1_ebtimeF_dtTrue_test_0>>>>>>>>>>>>>>>>>>>>>>>>>>
TRAIN
2010
TEST
503
TEST
503
	iters: 100, epoch: 1 | loss: 0.8623445
	speed: 0.4053s/iter; left time: 10172.8312s
	iters: 200, epoch: 1 | loss: 0.9460554
	speed: 0.0193s/iter; left time: 483.6492s
Epoch: 1 cost time: 44.913442611694336
Epoch: 1, Steps: 252 | Train Loss: 0.770 Vali Loss: 0.711 Vali Acc: 0.491 Test Loss: 0.711 Test Acc: 0.491
Validation loss decreased (inf --> -0.491054).  Saving model ...
	iters: 100, epoch: 2 | loss: 0.5215203
	speed: 0.6308s/iter; left time: 15675.5331s
	iters: 200, epoch: 2 | loss: 0.6149132
	speed: 0.0188s/iter; left time: 465.2888s
Epoch: 2 cost time: 41.04094958305359
Epoch: 2, Steps: 252 | Train Loss: 0.763 Vali Loss: 0.702 Vali Acc: 0.477 Test Loss: 0.702 Test Acc: 0.477
EarlyStopping counter: 1 out of 3
	iters: 100, epoch: 3 | loss: 0.6710754
	speed: 0.6240s/ite